In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression


def collect_prices_and_returns(tickers, time_period):
    df = yf.download(tickers, period = time_period)
    df_close = df.loc[:, "Adj Close"]
    df_returns = df_close.pct_change().dropna()
    df_returns.dropna(inplace = True)
    return df_returns

t1 = "AAPL"
t2 = "MSFT"
idx = "^GSPC"

df_returns = collect_prices_and_returns([t1, t2, idx], time_period = "3y")
df_returns_log = df_returns.apply(lambda x: np.log(1 + x))
df_weekly_returns_log = df_returns_log.reset_index().groupby(pd.Grouper(key = 'Date', freq = "W")).sum()
df_weekly_returns  = df_weekly_returns_log.apply(lambda x: np.exp(x) - 1)
df_monthly_returns_log = df_returns_log.reset_index().groupby(pd.Grouper(key = 'Date', freq = "ME")).sum()
df_monthly_returns  = df_monthly_returns_log.apply(lambda x: np.exp(x) - 1)

df_returns.head()

frequency = "daily"
if frequency == "daily":
    returns_ts = df_returns
elif frequency == "weekly":
    returns_ts = df_weekly_returns
elif frequency == "monthly":
    returns_ts = df_monthly_returns

corr_mat = returns_ts.loc[:, [t1, t2]].corr()
corr_mat_sp = returns_ts.loc[:, [t1, t2]].corr(method = "spearman")

[*********************100%%**********************]  3 of 3 completed


In [26]:
X = returns_ts.loc[:, [idx]]
y1 = returns_ts.loc[:, t1]
y2 = returns_ts.loc[:, t2]

lr1 = LinearRegression()
lr1.fit(X, y1)

lr2 = LinearRegression()
lr2.fit(X, y2)

beta_1 = lr1.coef_[0]
beta_2 = lr2.coef_[0]
sigma_f2 = returns_ts.loc[:, idx].var()

e1 = y1 - lr1.predict(X)
e2 = y2 - lr2.predict(X)

e = pd.DataFrame(dict(e1 = e1, e2 = e2))
var1 = beta_1**2 * sigma_f2 + e1.var()
var2 = beta_2**2 * sigma_f2 + e2.var()

corr_f = beta_1 * beta_2 * sigma_f2 / (np.sqrt(var1) * np.sqrt(var2))
corr_f

0.6315312751396065

In [24]:
returns_ts.var()

Ticker
AAPL     0.000288
MSFT     0.000299
^GSPC    0.000119
dtype: float64

In [4]:
import numpy as np
df_returns_log = df_returns.apply(lambda x: np.log(1 + x))
df_weekly_returns_log = df_returns_log.reset_index().groupby(pd.Grouper(key = 'Date', freq = "W")).sum()
df_weekly_returns  = df_weekly_returns_log.apply(lambda x: np.exp(x) - 1)

df_monthly_returns_log = df_returns_log.reset_index().groupby(pd.Grouper(key = 'Date', freq = "ME")).sum()
df_monthly_returns  = df_monthly_returns_log.apply(lambda x: np.exp(x) - 1)


In [6]:
df_monthly_returns

Ticker,AAPL,MSFT,^GSPC
Date,,,
2021-06-30,0.087934,0.080187,0.015984
2021-07-31,0.064983,0.051716,0.022748
2021-08-31,0.042489,0.061591,0.028990
2021-09-30,-0.068037,-0.066119,-0.047569
2021-10-31,0.058657,0.176291,0.069144
2021-11-30,0.105082,-0.001282,-0.008334
2021-12-31,0.074229,0.017333,0.043613
2022-01-31,-0.015712,-0.075345,-0.052585
2022-02-28,-0.054066,-0.037212,-0.031360
